# Temporal and Spatial Data Visualization
This is a personal practice of data visulization with various python libraries and open access data.

@ Shunan Feng

## 1. Global Annual Temperature Anomaly Interactive scatter plots with lowess trendlines

Reference:
https://plotly.com/python/linear-fits/ 

Data source: NASA's Goddard Institute for Space Studies (GISS). Credit: NASA/GISS
https://climate.nasa.gov/vital-signs/global-temperature/

In [1]:
import plotly.express as px
import pandas as pd

df = pd.read_table(r'https://data.giss.nasa.gov/gistemp/graphs/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.txt',
                   skiprows=[0,1,2,4], sep='\s+')
df = df.rename(columns={"No_Smoothing": "temperature anomaly", "Lowess(5)": "lowess"})
fig = px.scatter(df, x="Year", y="temperature anomaly", trendline="lowess")
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Temperature Anomaly (\u2103)")
fig.update_xaxes(rangeslider_visible=True)
fig.show()


## 2. COVID-19 
### 2.1 Total tests for COVID 19
Spatial bubble map of total number of testing for COVID-19.

- Map with plotly: https://plotly.com/python/bubble-maps/;
- Testing for COVID-19: this data is collected by the Our World in Data team from official reports.
https://github.com/owid/covid-19-data/tree/master/public/data
- Note: time slider is temporarily disabled since the timestamp didn't match in the dataset.
 

In [2]:
import pandas as pd
import plotly.express as px

df = pd.read_excel('https://covid.ourworldindata.org/data/owid-covid-data.xlsx')
df = df.dropna(subset=['total_tests_per_thousand'])    

fig = px.scatter_geo(df, locations="iso_code", color="location",
                     hover_name="location", size="total_tests_per_thousand",
                    #  animation_group="location", animation_frame="date", 
                     projection="natural earth")
fig.update_layout(title_text="Total Tests per Thousand People")
fig.show()


### 2.2 Global Total confirmed cases
- Plot with altair: https://altair-viz.github.io/gallery/airports_count.html
- Data: COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University https://github.com/CSSEGISandData/COVID-19
- Note: the coordinates is calculated by taking the average so the location of some countries may lie outside its main continental area.

In [4]:
import altair as alt
import pandas as pd
from vega_datasets import data

urlcovid = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df = pd.read_csv(urlcovid)
dfcoor = df.groupby('Country/Region').mean()
df = df.groupby('Country/Region').sum()
df['sum'] = df.iloc[:,-1]
df['latitude'] = dfcoor['Lat']
df['longitude'] = dfcoor['Long']

# prepare background map
source = alt.topo_feature(data.world_110m.url, 'countries')
background = alt.Chart(source).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=600, height=400
).project('naturalEarth1')
# plot point data
points = alt.Chart(df).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.Size('sum:Q', title='Number of Confirmed Cases'),
    color=alt.value('steelblue'),
    tooltip=['countries:N','sum:Q']
).properties(
    title='Global Confirmed COVID-19 Cases'
)
background + points

alt.LayerChart(...)